# 도보

In [12]:
import requests

# ✅ 주소 → 위도/경도 변환
def address_to_coord(address, app_key):
    url = "https://apis.openapi.sk.com/tmap/geo/fullAddrGeo"
    params = {
        "version": 1,
        "format": "json",
        "fullAddr": address,
        "appKey": app_key
    }
    res = requests.get(url, params=params)
    data = res.json()

    try:
        coord = data["coordinateInfo"]["coordinate"][0]
        return float(coord["lat"]), float(coord["lon"])
    except:
        print(f"❌ 주소 변환 실패: {address}")
        return None, None

# ✅ 도보 경로 안내
def get_tmap_pedestrian_route(start_lat, start_lon, end_lat, end_lon, start_name, end_name, app_key):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian"
    headers = {
        "appKey": app_key,
        "Content-Type": "application/json"
    }

    body = {
        "startX": str(start_lon),
        "startY": str(start_lat),
        "endX": str(end_lon),
        "endY": str(end_lat),
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "startName": start_name,
        "endName": end_name
    }

    res = requests.post(url, json=body, headers=headers)
    data = res.json()

    if 'features' not in data:
        print("❌ 경로 요청 실패:", data)
        return

    # ✅ 요약 정보
    total_time_sec = data["features"][0]["properties"]["totalTime"]
    total_time_min = total_time_sec // 60
    crosswalk_count = sum(1 for f in data['features'] if '횡단보도' in f['properties'].get('description', ''))

    print(f"\n📍 출발지: {start_name}")
    print(f"📍 도착지: {end_name}")
    print(f"\n🕒 예상 소요 시간: {total_time_min}분")
    print(f"🚸 건너야 할 횡단보도 수: {crosswalk_count}개")

    print(f"\n🧭 상세 경로 안내:")
    step_num = 1
    for step in data['features']:
        desc = step['properties'].get('description')
        if desc:
            print(f"{step_num}. {desc}")
            step_num += 1


In [ ]:
if __name__ == "__main__":
    app_key = "abcdef1234567890abcdef1234"

    start_name = input("출발지를 입력하세요 (예: 서울역): ")
    end_name = input("도착지를 입력하세요 (예: 광운대): ")

    start_lat, start_lon = address_to_coord(start_name, app_key)
    end_lat, end_lon = address_to_coord(end_name, app_key)

    if start_lat and end_lat:
        get_tmap_pedestrian_route(start_lat, start_lon, end_lat, end_lon, start_name, end_name, app_key)


📍 출발지: 광운대학교
📍 도착지: 석계역

🕒 예상 소요 시간: 14분
🚸 건너야 할 횡단보도 수: 2개

🧭 상세 경로 안내:
1. 보행자도로 을 따라 15m 이동
2. 보행자도로, 15m
3. 좌회전 후 보행자도로 을 따라 209m 이동 
4. 보행자도로, 209m
5. 광운대학교우체국 에서 좌회전 후 광운로 을 따라 126m 이동 
6. 광운로, 126m
7. 우측 횡단보도 후 14m 이동 
8. , 14m
9. 우동김밥이야기 에서 좌측 횡단보도 후 보행자도로 을 따라 16m 이동 
10. 보행자도로, 16m
11. 광운로, 51m
12. 직진 후 93m 이동 
13. , 93m
14. 좌회전 후 한천로 을 따라 470m 이동 
15. 한천로, 470m
16. 좌회전 후 99m 이동 
17. , 99m
18. 우회전 후 보행자도로 을 따라 16m 이동 
19. 보행자도로, 16m
20. 도착


# 대중교통

In [10]:
import requests
import re

# ✅ Google 주소 → 위경도 변환
def address_to_coord_google(address, google_api_key):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": google_api_key}
    res = requests.get(url, params=params).json()
    try:
        loc = res["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    except:
        print(f"❌ 주소 변환 실패: {address}")
        return None, None

# ✅ Tmap 보행자 안내 요청
def get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, app_key):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian"
    headers = {
        "appKey": app_key,
        "Content-Type": "application/json"
    }
    body = {
        "startX": str(start_lon),
        "startY": str(start_lat),
        "endX": str(end_lon),
        "endY": str(end_lat),
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "startName": "출발지",
        "endName": "도착지"
    }
    res = requests.post(url, headers=headers, json=body).json()
    steps = []
    if "features" not in res:
        print("❌ Tmap 경로 요청 실패:", res)
        return steps
    for feat in res["features"]:
        desc = feat["properties"].get("description")
        if desc:
            steps.append(desc)
    return steps

# ✅ Google Directions + Tmap 보행자 통합 안내
def get_combined_route_script_cleaned(start_lat, start_lon, end_lat, end_lon, google_api_key, tmap_key, start_name, end_name):
    g_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{start_lat},{start_lon}",
        "destination": f"{end_lat},{end_lon}",
        "mode": "transit",
        "language": "ko",
        "key": google_api_key
    }
    res = requests.get(g_url, params=params).json()
    if res["status"] != "OK":
        print("❌ Google 경로 요청 실패:", res["status"])
        return []

    leg = res["routes"][0]["legs"][0]
    duration = leg["duration"]["text"]
    total_script = []
    bus_lines = []

    # 상단 요약
    for step in leg["steps"]:
        if step["travel_mode"] == "TRANSIT":
            line = step["transit_details"]["line"].get("short_name", step["transit_details"]["line"].get("name"))
            if line not in bus_lines:
                bus_lines.append(line)

    summary = f"{start_name}부터 {end_name}까지 총 소요시간은 {duration}입니다. "
    if bus_lines:
        summary += "탑승할 버스는 " + ", ".join([f"{b}번" for b in bus_lines]) + "입니다. "
    summary += "안내를 시작합니다."
    total_script.append(summary)

    # 세부 안내
    step_num = 1
    for step in leg["steps"]:
        mode = step["travel_mode"]
        dist = step["distance"]["text"]
        dur = step["duration"]["text"]
        if mode == "WALKING":
            s = step["start_location"]
            e = step["end_location"]
            tmap_steps = get_tmap_pedestrian_steps(s["lat"], s["lng"], e["lat"], e["lng"], tmap_key)
            for desc in tmap_steps:
                # 중복되지 않도록 명확한 안내 문장만 출력
                if not re.fullmatch(r'[.,\s\d\w가-힣]*[,.]\s*\d+m', desc):  # 거리만 있는 짧은 문장은 제외
                    total_script.append(f"{step_num}. {desc}")
                    step_num += 1
        elif mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"].get("short_name", t["line"].get("name", "노선 없음"))
            vehicle = t["line"]["vehicle"]["name"]
            dep = t["departure_stop"]["name"]
            arr = t["arrival_stop"]["name"]
            stops = t["num_stops"]
            total_script.append(
                f"{step_num}. {dep}에서 {line}번 {vehicle}을 탑승하여 {stops} 정거장({dur}) 이동 후 {arr}에서 하차합니다."
            )
            step_num += 1

    return total_script



In [ ]:
if __name__ == "__main__":
    google_api_key = "abcdef1234567890abcdef1234"
    tmap_api_key = "abcdef1234567890abcdef1234"

    start_name = input("출발지를 입력하세요 (예: 서울역): ")
    end_name = input("도착지를 입력하세요 (예: 광운대): ")

    start_lat, start_lon = address_to_coord_google(start_name, google_api_key)
    end_lat, end_lon = address_to_coord_google(end_name, google_api_key)

   
script = get_combined_route_script_cleaned(
    start_lat, start_lon, end_lat, end_lon,
    google_api_key, tmap_api_key,
    start_name, end_name
)

for line in script:
    print(line)

서울역부터 광운대까지 총 소요시간은 1시간 18분입니다. 탑승할 버스는 N16번, 1218번입니다. 안내를 시작합니다.
1. 통일로 을 따라 139m 이동
2. 서울역유료주차장 에서 좌측 횡단보도 후 62m 이동 
3. 좌회전 후 보행자도로 을 따라 73m 이동 
4. 도착
5. 서울역버스환승센터 (5번승강장)에서 N16번 버스을 탑승하여 23 정거장(44분) 이동 후 수유역.강북구청에서 하차합니다.
6. 수유역.강북구청에서 1218번 버스을 탑승하여 12 정거장(12분) 이동 후 장위3동주민센터에서 하차합니다.
7. 한천로 을 따라 47m 이동
8. 영신교회 에서 우측 횡단보도 후 보행자도로 을 따라 28m 이동 
9. 직진 후 장위로 을 따라 5m 이동 
10. 횡단보도 후 보행자도로 을 따라 5m 이동 
11. 둘둘치킨 에서 좌회전 후 한천로 을 따라 93m 이동 
12. 직진 후 광운로 을 따라 10m 이동 
13. 피노키오문구 에서 10시 방향 횡단보도 후 16m 이동 
14. 우동김밥이야기 에서 우측 횡단보도 후 보행자도로 을 따라 14m 이동 
15. 좌회전 후 광운로 을 따라 126m 이동 
16. 광운대학교우체국 에서 우회전 후 보행자도로 을 따라 159m 이동 
17. 도착


In [21]:
import requests
import re

# ✅ Google 주소 → 위경도 변환
def address_to_coord_google(address, google_api_key):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": google_api_key}
    res = requests.get(url, params=params).json()
    try:
        loc = res["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    except:
        print(f"❌ 주소 변환 실패: {address}")
        return None, None

# ✅ Tmap 보행자 안내 요청
def get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, app_key):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian"
    headers = {
        "appKey": app_key,
        "Content-Type": "application/json"
    }
    body = {
        "startX": str(start_lon),
        "startY": str(start_lat),
        "endX": str(end_lon),
        "endY": str(end_lat),
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "startName": "출발지",
        "endName": "도착지"
    }
    res = requests.post(url, headers=headers, json=body).json()
    steps = []
    if "features" not in res:
        print("❌ Tmap 경로 요청 실패:", res)
        return steps
    for feat in res["features"]:
        desc = feat["properties"].get("description")
        if desc:
            steps.append(desc)
    return steps

# ✅ Google + Tmap 통합 대중교통 안내 (TTS 대본용)
def get_combined_route_script_cleaned(start_lat, start_lon, end_lat, end_lon, google_api_key, tmap_key, start_name, end_name):
    g_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{start_lat},{start_lon}",
        "destination": f"{end_lat},{end_lon}",
        "mode": "transit",
        "language": "ko",
        "key": google_api_key
    }
    res = requests.get(g_url, params=params).json()
    if res["status"] != "OK":
        print("❌ Google 경로 요청 실패:", res["status"])
        return []

    leg = res["routes"][0]["legs"][0]
    duration = leg["duration"]["text"]
    total_script = []
    bus_lines = []

    for step in leg["steps"]:
        if step["travel_mode"] == "TRANSIT":
            line = step["transit_details"]["line"].get("short_name", step["transit_details"]["line"].get("name"))
            if line not in bus_lines:
                bus_lines.append(line)

    summary = f"{start_name}부터 {end_name}까지 총 소요시간은 {duration}입니다. "
    if bus_lines:
        summary += "탑승할 버스는 " + ", ".join([f"{b}번" for b in bus_lines]) + "입니다. "
    summary += "안내를 시작합니다."
    total_script.append(summary)

    step_num = 1
    for step in leg["steps"]:
        mode = step["travel_mode"]
        dist = step["distance"]["text"]
        dur = step["duration"]["text"]
        if mode == "WALKING":
            s = step["start_location"]
            e = step["end_location"]
            tmap_steps = get_tmap_pedestrian_steps(s["lat"], s["lng"], e["lat"], e["lng"], tmap_key)
            for desc in tmap_steps:
                if not re.fullmatch(r'[.,\s\d\w가-힣]*[,.]\s*\d+m', desc):
                    total_script.append(f"{step_num}. {desc}")
                    step_num += 1
        elif mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"].get("short_name", t["line"].get("name", "노선 없음"))
            vehicle = t["line"]["vehicle"]["name"]
            dep = t["departure_stop"]["name"]
            arr = t["arrival_stop"]["name"]
            stops = t["num_stops"]
            total_script.append(
                f"{step_num}. {dep}에서 {line}번 {vehicle}을 탑승하여 {stops} 정거장({dur}) 이동 후 {arr}에서 하차합니다."
            )
            step_num += 1

    return total_script

# ✅ 전체 실행 흐름
def run_combined_route_system_fixed(google_api_key, tmap_api_key):
    print("어떤 방식으로 이동할까요?")
    print("1. 대중교통 (Google + Tmap 통합)")
    print("2. 도보 (Tmap API 사용)")
    mode_choice = input("선택 (1 또는 2): ").strip()

    # 출발지/도착지 입력은 한 번만
    start_name = input("출발지를 입력하세요 (예: 서울역): ").strip()
    end_name = input("도착지를 입력하세요 (예: 광운대): ").strip()

    # 위경도 변환
    start_lat, start_lon = address_to_coord_google(start_name, google_api_key)
    end_lat, end_lon = address_to_coord_google(end_name, google_api_key)

    if not start_lat or not end_lat:
        print("❌ 주소 좌표 변환 실패")
        return

    if mode_choice == "1":
        print(f"\n📍 {start_name}부터 {end_name}까지 대중교통 경로 안내를 시작합니다.")
        script = get_combined_route_script_detailed(
            start_lat, start_lon, end_lat, end_lon,
            google_api_key, tmap_api_key,
            start_name, end_name
        )
        for line in script:
            print(line)
    elif mode_choice == "2":
        print(f"\n📍 {start_name}부터 {end_name}까지 도보 경로 안내를 시작합니다.")
        steps = get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, tmap_api_key)
        if steps:
            print(f"\n🕒 예상 소요시간 및 상세 경로 안내:")
            for i, desc in enumerate(steps, 1):
                print(f"{i}. {desc}")
        else:
            print("❌ 도보 경로 안내 실패")
    else:
        print("❌ 잘못된 선택입니다. 1 또는 2를 입력하세요.")

        
def get_combined_route_script_detailed(start_lat, start_lon, end_lat, end_lon, google_api_key, tmap_key, start_name, end_name):
    g_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{start_lat},{start_lon}",
        "destination": f"{end_lat},{end_lon}",
        "mode": "transit",
        "language": "ko",
        "key": google_api_key
    }
    res = requests.get(g_url, params=params).json()
    if res["status"] != "OK":
        print("❌ Google 경로 요청 실패:", res["status"])
        return []

    leg = res["routes"][0]["legs"][0]
    duration = leg["duration"]["text"]
    total_script = []
    bus_lines = []

    # 상단 요약
    for step in leg["steps"]:
        if step["travel_mode"] == "TRANSIT":
            line = step["transit_details"]["line"].get("short_name", step["transit_details"]["line"].get("name"))
            if line not in bus_lines:
                bus_lines.append(line)

    summary = f"{start_name}부터 {end_name}까지 총 소요시간은 {duration}입니다. "
    if bus_lines:
        summary += "탑승할 버스는 " + ", ".join([f"{b}번" for b in bus_lines]) + "입니다. "
    summary += "안내를 시작합니다."
    total_script.append(summary)

    # 세부 안내
    step_num = 1
    for step in leg["steps"]:
        mode = step["travel_mode"]
        dist = step["distance"]["text"]
        dur = step["duration"]["text"]
        if mode == "WALKING":
            s = step["start_location"]
            e = step["end_location"]
            tmap_steps = get_tmap_pedestrian_steps(s["lat"], s["lng"], e["lat"], e["lng"], tmap_key)
            for desc in tmap_steps:
                desc_clean = desc.strip()
                if not re.fullmatch(r'[.,\s\d\w가-힣]*[,.]\s*\d+m', desc_clean) and desc_clean:
                    total_script.append(f"{step_num}. {desc_clean} ({dist}, {dur})")
                    step_num += 1
        elif mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"].get("short_name", t["line"].get("name", "노선 없음"))
            vehicle = t["line"]["vehicle"]["name"]
            dep = t["departure_stop"]["name"]
            arr = t["arrival_stop"]["name"]
            dep_time = t["departure_time"]["text"]
            arr_time = t["arrival_time"]["text"]
            stops = t["num_stops"]
            total_script.append(
                f"{step_num}. {dep} 정류장에서 {line}번 {vehicle}을({dep_time} 출발) 탑승하여 {stops}개 정거장을 이동하고, {arr_time}에 {arr} 정류장에서 하차합니다. 예상 소요 시간은 {dur}입니다."
            )
            step_num += 1

    return total_script


# 통합

In [ ]:
import webbrowser
import os

def generate_map_html(start_lat, start_lon, end_lat, end_lon, waypoints=None):
    html_template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>경로 시각화</title>
        <meta charset="utf-8">
        <style>
            #map {{
                height: 100vh;
                width: 100%;
            }}
        </style>
        <script src="https://maps.googleapis.com/maps/api/js?key=abcdef1234567890abcdef1234&libraries=places"></script>
        <script>
            function initMap() {{
                const start = {{ lat: {start_lat}, lng: {start_lon} }};
                const end = {{ lat: {end_lat}, lng: {end_lon} }};
                const map = new google.maps.Map(document.getElementById('map'), {{
                    zoom: 14,
                    center: start
                }});

                const directionsService = new google.maps.DirectionsService();
                const directionsRenderer = new google.maps.DirectionsRenderer();
                directionsRenderer.setMap(map);

                directionsService.route(
                    {{
                        origin: start,
                        destination: end,
                        travelMode: google.maps.TravelMode.TRANSIT
                    }},
                    (response, status) => {{
                        if (status === "OK") {{
                            directionsRenderer.setDirections(response);
                        }} else {{
                            alert("경로 요청 실패: " + status);
                        }}
                    }}
                );
            }}
        </script>
    </head>
    <body onload="initMap()">
        <div id="map"></div>
    </body>
    </html>
    """

    # 저장 및 실행
    html_path = "route_map.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_template)
    
    full_path = os.path.abspath(html_path)
    webbrowser.open(f"file://{full_path}")


In [10]:
import requests
import re

# ✅ Google 주소 → 위경도 변환
def address_to_coord_google(address, google_api_key):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": google_api_key}
    res = requests.get(url, params=params).json()
    try:
        loc = res["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    except:
        print(f"❌ 주소 변환 실패: {address}")
        return None, None

# ✅ Tmap 보행자 안내 요청
def get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, app_key):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian"
    headers = {
        "appKey": app_key,
        "Content-Type": "application/json"
    }
    body = {
        "startX": str(start_lon),
        "startY": str(start_lat),
        "endX": str(end_lon),
        "endY": str(end_lat),
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "startName": "출발지",
        "endName": "도착지"
    }
    res = requests.post(url, headers=headers, json=body).json()
    steps = []
    if "features" not in res:
        print("❌ Tmap 경로 요청 실패:", res)
        return steps
    for feat in res["features"]:
        desc = feat["properties"].get("description")
        if desc:
            steps.append(desc)
    return steps

# ✅ Google + Tmap 통합 대중교통 안내 (TTS 대본용)
def get_combined_route_script_cleaned(start_lat, start_lon, end_lat, end_lon, google_api_key, tmap_key, start_name, end_name):
    g_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{start_lat},{start_lon}",
        "destination": f"{end_lat},{end_lon}",
        "mode": "transit",
        "language": "ko",
        "key": google_api_key
    }
    res = requests.get(g_url, params=params).json()
    if res["status"] != "OK":
        print("❌ Google 경로 요청 실패:", res["status"])
        return []

    leg = res["routes"][0]["legs"][0]
    duration = leg["duration"]["text"]
    total_script = []
    bus_lines = []

    for step in leg["steps"]:
        if step["travel_mode"] == "TRANSIT":
            line = step["transit_details"]["line"].get("short_name", step["transit_details"]["line"].get("name"))
            if line not in bus_lines:
                bus_lines.append(line)

    summary = f"{start_name}부터 {end_name}까지 총 소요시간은 {duration}입니다. "
    if bus_lines:
        summary += "탑승할 버스는 " + ", ".join([f"{b}번" for b in bus_lines]) + "입니다. "
    summary += "안내를 시작합니다."
    total_script.append(summary)

    step_num = 1
    for step in leg["steps"]:
        mode = step["travel_mode"]
        dist = step["distance"]["text"]
        dur = step["duration"]["text"]
        if mode == "WALKING":
            s = step["start_location"]
            e = step["end_location"]
            tmap_steps = get_tmap_pedestrian_steps(s["lat"], s["lng"], e["lat"], e["lng"], tmap_key)
            for desc in tmap_steps:
                if not re.fullmatch(r'[.,\s\d\w가-힣]*[,.]\s*\d+m', desc):
                    total_script.append(f"{step_num}. {desc}")
                    step_num += 1
        elif mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"].get("short_name", t["line"].get("name", "노선 없음"))
            vehicle = t["line"]["vehicle"]["name"]
            dep = t["departure_stop"]["name"]
            arr = t["arrival_stop"]["name"]
            stops = t["num_stops"]
            total_script.append(
                f"{step_num}. {dep}에서 {line}번 {vehicle}을 탑승하여 {stops} 정거장({dur}) 이동 후 {arr}에서 하차합니다."
            )
            step_num += 1

    return total_script

# ✅ 전체 실행 흐름
def run_combined_route_system_fixed(google_api_key, tmap_api_key):
    print("어떤 방식으로 이동할까요?")
    print("1. 대중교통 (Google + Tmap 통합)")
    print("2. 도보 (Tmap API 사용)")
    mode_choice = input("선택 (1 또는 2): ").strip()

    # 출발지/도착지 입력은 한 번만
    start_name = input("출발지를 입력하세요 (예: 서울역): ").strip()
    end_name = input("도착지를 입력하세요 (예: 광운대): ").strip()

    # 위경도 변환
    start_lat, start_lon = address_to_coord_google(start_name, google_api_key)
    end_lat, end_lon = address_to_coord_google(end_name, google_api_key)

    if not start_lat or not end_lat:
        print("❌ 주소 좌표 변환 실패")
        return

    if mode_choice == "1":
        print(f"\n📍 {start_name}부터 {end_name}까지 대중교통 경로 안내를 시작합니다.")
        script = get_combined_route_script_detailed(
            start_lat, start_lon, end_lat, end_lon,
            google_api_key, tmap_api_key,
            start_name, end_name
        )
        for line in script:
            print(line)
    elif mode_choice == "2":
        print(f"\n📍 {start_name}부터 {end_name}까지 도보 경로 안내를 시작합니다.")
        steps = get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, tmap_api_key)
        if steps:
            print(f"\n🕒 예상 소요시간 및 상세 경로 안내:")
            for i, desc in enumerate(steps, 1):
                print(f"{i}. {desc}")
        else:
            print("❌ 도보 경로 안내 실패")
    else:
        print("❌ 잘못된 선택입니다. 1 또는 2를 입력하세요.")

        
def get_combined_route_script_detailed(start_lat, start_lon, end_lat, end_lon, google_api_key, tmap_key, start_name, end_name):
    g_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{start_lat},{start_lon}",
        "destination": f"{end_lat},{end_lon}",
        "mode": "transit",
        "language": "ko",
        "key": google_api_key
    }
    res = requests.get(g_url, params=params).json()
    if res["status"] != "OK":
        print("❌ Google 경로 요청 실패:", res["status"])
        return []

    leg = res["routes"][0]["legs"][0]
    duration = leg["duration"]["text"]
    total_script = []
    bus_lines = []

    # 상단 요약
    for step in leg["steps"]:
        if step["travel_mode"] == "TRANSIT":
            line = step["transit_details"]["line"].get("short_name", step["transit_details"]["line"].get("name"))
            if line not in bus_lines:
                bus_lines.append(line)

    summary = f"{start_name}부터 {end_name}까지 총 소요시간은 {duration}입니다. "
    if bus_lines:
        summary += "탑승할 버스는 " + ", ".join([f"{b}번" for b in bus_lines]) + "입니다. "
    summary += "안내를 시작합니다."
    total_script.append(summary)

    # 세부 안내
    step_num = 1
    for step in leg["steps"]:
        mode = step["travel_mode"]
        dist = step["distance"]["text"]
        dur = step["duration"]["text"]
        if mode == "WALKING":
            s = step["start_location"]
            e = step["end_location"]
            tmap_steps = get_tmap_pedestrian_steps(s["lat"], s["lng"], e["lat"], e["lng"], tmap_key)
            for desc in tmap_steps:
                desc_clean = desc.strip()
                if not re.fullmatch(r'[.,\s\d\w가-힣]*[,.]\s*\d+m', desc_clean) and desc_clean:
                    total_script.append(f"{step_num}. {desc_clean} ({dist}, {dur})")
                    step_num += 1
        elif mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"].get("short_name", t["line"].get("name", "노선 없음"))
            vehicle = t["line"]["vehicle"]["name"]
            dep = t["departure_stop"]["name"]
            arr = t["arrival_stop"]["name"]
            dep_time = t["departure_time"]["text"]
            arr_time = t["arrival_time"]["text"]
            stops = t["num_stops"]
            total_script.append(
                f"{step_num}. {dep} 정류장에서 {line}번 {vehicle}을({dep_time} 출발) 탑승하여 {stops}개 정거장을 이동하고, {arr_time}에 {arr} 정류장에서 하차합니다. 예상 소요 시간은 {dur}입니다."
            )
            step_num += 1

    return total_script

def get_tmap_pedestrian_script(start_lat, start_lon, end_lat, end_lon, app_key, start_name, end_name):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian"
    headers = {
        "appKey": app_key,
        "Content-Type": "application/json"
    }
    body = {
        "startX": str(start_lon),
        "startY": str(start_lat),
        "endX": str(end_lon),
        "endY": str(end_lat),
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "startName": "출발지",
        "endName": "도착지"
    }
    res = requests.post(url, headers=headers, json=body).json()
    script = []

    if "features" not in res:
        print("❌ Tmap 경로 요청 실패:", res)
        return script

    # 총 소요 시간
    total_time_sec = res["features"][0]["properties"].get("totalTime", 0)
    total_time_min = total_time_sec // 60
    script.append(f"{start_name}부터 {end_name}까지 도보로 이동합니다. 예상 소요 시간은 약 {total_time_min}분입니다. 안내를 시작합니다.")

    step_num = 1
    for feat in res["features"]:
        desc = feat["properties"].get("description", "").strip()

        if (
            not desc or
            re.fullmatch(r'[.,\s]*', desc) or
            re.fullmatch(r'[가-힣a-zA-Z0-9\s.,]*[,]\s*\d+m', desc)
        ):
            continue

        script.append(f"{step_num}. {desc}")
        step_num += 1

    return script

def run_combined_route_system_tts(google_api_key, tmap_api_key):
    start_name = input("출발지를 입력하세요 (예: 서울역): ").strip()
    end_name = input("도착지를 입력하세요 (예: 광운대): ").strip()

    mode_choice = input("어떤 방식으로 이동할까요? (1. 대중교통 or 2. 도보) ").strip()

    start_lat, start_lon = address_to_coord_google(start_name, google_api_key)
    end_lat, end_lon = address_to_coord_google(end_name, google_api_key)

    if not start_lat or not end_lat:
        print("❌ 주소 좌표 변환 실패")
        return

    # 🔍 시각화 호출
    generate_map_html(start_lat, start_lon, end_lat, end_lon)

    if mode_choice == "1":
        print(f"\n📍 {start_name}부터 {end_name}까지 대중교통 경로 안내를 시작합니다.")
        script = get_combined_route_script_detailed(
            start_lat, start_lon, end_lat, end_lon,
            google_api_key, tmap_api_key,
            start_name, end_name
        )
        for line in script:
            print(line)

    elif mode_choice == "2":
        print(f"\n📍 {start_name}부터 {end_name}까지 도보 경로 안내를 시작합니다.")
        script = get_tmap_pedestrian_script(
            start_lat, start_lon, end_lat, end_lon,
            tmap_api_key, start_name, end_name
        )
        for line in script:
            print(line)
    else:
        print("❌ 잘못된 선택입니다. 1 또는 2를 입력하세요.")


In [ ]:
google_api_key = "abcdef1234567890abcdef1234"
tmap_api_key = "abcdef1234567890abcdef1234"
   
run_combined_route_system_tts(google_api_key, tmap_api_key)


📍 광운대 누리관부터 월계1동 주민센터까지 대중교통 경로 안내를 시작합니다.
광운대 누리관부터 월계1동 주민센터까지 총 소요시간은 17분입니다. 탑승할 버스는 261번입니다. 안내를 시작합니다.
1. 16m 이동 (62 m, 1분)
2. 광운대학교 누리관 에서 좌측 횡단보도 후 보행자도로 을 따라 13m 이동 (62 m, 1분)
3. 8시 방향 좌회전 후 78m 이동 (62 m, 1분)
4. 도착 (62 m, 1분)
5. 벼루말교 정류장에서 261번 버스을(오후 7:24 출발) 탑승하여 5개 정거장을 이동하고, 오후 7:28에 월계현대아파트 정류장에서 하차합니다. 예상 소요 시간은 4분입니다.
6. 석계로 을 따라 83m 이동 (84 m, 1분)
7. 꿈꾸는 작은도서관 에서 좌회전 후 보행자도로 을 따라 12m 이동 (84 m, 1분)
8. 도착 (84 m, 1분)


# 실시간 경로(좌표값 기반)

In [75]:
import requests
import re
import time
import threading
import math

# 시뮬레이션된 GPS 경로 (실제 GPS 센서 대신 사용)
SIMULATED_PATH = [
    (37.623955, 127.061509),  # 시작점
    (37.618532, 127.058462),  # 중간점 1
    (37.615391, 127.057862),  # 중간점 2  
    (37.601913, 127.055891),  # 중간점 3
    (37.602240, 127.055203),  # 도착점
]

class NavigationSystem:
    def __init__(self, google_api_key, tmap_api_key):
        self.google_api_key = google_api_key
        self.tmap_api_key = tmap_api_key
        self.route_steps = []  # 경로 단계별 안내 텍스트
        self.route_coordinates = []  # 경로 단계별 좌표
        self.current_step_index = 0
        self.current_gps_index = 0
        self.navigation_active = False
        self.gps_thread = None
        self.last_announced_step = -1  # 마지막으로 안내한 단계 추적
        
    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """두 좌표 간의 거리를 미터 단위로 계산"""
        R = 6371000  # 지구 반지름 (미터)
        
        lat1_rad = math.radians(lat1)
        lat2_rad = math.radians(lat2)
        delta_lat = math.radians(lat2 - lat1)
        delta_lon = math.radians(lon2 - lon1)
        
        a = (math.sin(delta_lat/2) * math.sin(delta_lat/2) +
             math.cos(lat1_rad) * math.cos(lat2_rad) *
             math.sin(delta_lon/2) * math.sin(delta_lon/2))
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        
        return R * c

    def address_to_coord_google(self, address):
        """Google 주소 → 위경도 변환"""
        url = "https://maps.googleapis.com/maps/api/geocode/json"
        params = {"address": address, "key": self.google_api_key}
        res = requests.get(url, params=params).json()
        try:
            loc = res["results"][0]["geometry"]["location"]
            return loc["lat"], loc["lng"]
        except:
            print(f"❌ 주소 변환 실패: {address}")
            return None, None

    def get_tmap_pedestrian_route(self, start_lat, start_lon, end_lat, end_lon):
        """Tmap 보행자 경로 요청 (좌표 포함)"""
        url = "https://apis.openapi.sk.com/tmap/routes/pedestrian"
        headers = {
            "appKey": self.tmap_api_key,
            "Content-Type": "application/json"
        }
        body = {
            "startX": str(start_lon),
            "startY": str(start_lat),
            "endX": str(end_lon),
            "endY": str(end_lat),
            "reqCoordType": "WGS84GEO",
            "resCoordType": "WGS84GEO",
            "startName": "출발지",
            "endName": "도착지"
        }
        res = requests.post(url, headers=headers, json=body).json()
        
        if "features" not in res:
            print("❌ Tmap 경로 요청 실패:", res)
            return [], []
            
        steps = []
        coordinates = []
        
        for feat in res["features"]:
            props = feat["properties"]
            geom = feat["geometry"]
            
            # 안내 텍스트 추출
            desc = props.get("description", "").strip()
            if desc and not re.fullmatch(r'[.,\s\d\w가-힣]*[,.]\s*\d+m', desc):
                steps.append(desc)
                
                # 해당 단계의 좌표 추출
                if geom["type"] == "Point":
                    coords = geom["coordinates"]
                    coordinates.append((coords[1], coords[0]))  # lat, lon 순서
                elif geom["type"] == "LineString":
                    # 선분의 시작점 사용
                    coords = geom["coordinates"][0]
                    coordinates.append((coords[1], coords[0]))
                    
        return steps, coordinates

    def get_combined_route_with_coordinates(self, start_lat, start_lon, end_lat, end_lon, start_name, end_name):
        """Google + Tmap 통합 경로 (좌표 포함)"""
        g_url = "https://maps.googleapis.com/maps/api/directions/json"
        params = {
            "origin": f"{start_lat},{start_lon}",
            "destination": f"{end_lat},{end_lon}",
            "mode": "transit",
            "language": "ko",
            "key": self.google_api_key
        }
        res = requests.get(g_url, params=params).json()
        
        if res["status"] != "OK":
            print("❌ Google 경로 요청 실패:", res["status"])
            return [], []

        leg = res["routes"][0]["legs"][0]
        duration = leg["duration"]["text"]
        route_steps = []
        route_coordinates = []
        bus_lines = []

        # 버스 노선 정보 수집
        for step in leg["steps"]:
            if step["travel_mode"] == "TRANSIT":
                line = step["transit_details"]["line"].get("short_name", 
                       step["transit_details"]["line"].get("name"))
                if line and line not in bus_lines:
                    bus_lines.append(line)

        # 요약 정보
        summary = f"{start_name}부터 {end_name}까지 총 소요시간은 {duration}입니다. "
        if bus_lines:
            summary += "탑승할 버스는 " + ", ".join([f"{line}번" for line in bus_lines]) + "입니다. "
        summary += "안내를 시작합니다."
        
        route_steps.append(summary)
        route_coordinates.append((start_lat, start_lon))

        # 세부 경로 단계별 처리
        for step in leg["steps"]:
            mode = step["travel_mode"]
            start_loc = step["start_location"]
            end_loc = step["end_location"]
            
            if mode == "WALKING":
                # 도보 구간은 Tmap으로 세부 경로 가져오기
                walk_steps, walk_coords = self.get_tmap_pedestrian_route(
                    start_loc["lat"], start_loc["lng"], 
                    end_loc["lat"], end_loc["lng"]
                )
                route_steps.extend(walk_steps)
                route_coordinates.extend(walk_coords)
                
            elif mode == "TRANSIT":
                # 대중교통 구간
                t = step["transit_details"]
                line = t["line"].get("short_name", t["line"].get("name", "버스"))
                vehicle = t["line"]["vehicle"]["name"]
                dep_stop = t["departure_stop"]["name"]
                arr_stop = t["arrival_stop"]["name"]
                dep_time = t["departure_time"]["text"]
                arr_time = t["arrival_time"]["text"]
                stops = t["num_stops"]
                
                transit_desc = (f"{dep_stop} 정류장에서 {line}번 {vehicle}을({dep_time} 출발) "
                              f"탑승하여 {stops}개 정거장을 이동하고, {arr_time}에 {arr_stop} 정류장에서 하차합니다.")
                
                route_steps.append(transit_desc)
                route_coordinates.append((start_loc["lat"], start_loc["lng"]))
                route_coordinates.append((end_loc["lat"], end_loc["lng"]))

        return route_steps, route_coordinates

    def get_pedestrian_route_with_coordinates(self, start_lat, start_lon, end_lat, end_lon, start_name, end_name):
        """도보 경로 (좌표 포함)"""
        steps, coordinates = self.get_tmap_pedestrian_route(start_lat, start_lon, end_lat, end_lon)
        
        if not steps:
            return [], []
            
        # 시작 안내 추가
        intro = f"{start_name}부터 {end_name}까지 도보로 이동합니다. 안내를 시작합니다."
        route_steps = [intro] + steps
        route_coordinates = [(start_lat, start_lon)] + coordinates
        
        return route_steps, route_coordinates

    def find_current_step_index(self, current_lat, current_lon):
        """현재 위치에서 가장 적절한 단계 찾기 - 개선된 버전"""
        if not self.route_coordinates:
            return 0
            
        # 디버그 정보 출력
        print(f"🔍 현재 위치: ({current_lat:.6f}, {current_lon:.6f})")
        print(f"🔍 현재 단계 인덱스: {self.current_step_index}")
        
        # 모든 단계에 대해 거리 계산
        distances = []
        for i, (step_lat, step_lon) in enumerate(self.route_coordinates):
            distance = self.haversine_distance(current_lat, current_lon, step_lat, step_lon)
            distances.append((i, distance))
            if i <= self.current_step_index + 3:  # 현재 단계 근처만 출력
                print(f"   단계 {i}: 거리 {distance:.1f}m")
        
        # 거리 기준으로 정렬
        distances.sort(key=lambda x: x[1])
        
        # 현재 단계보다 앞선 단계는 제외하고, 가장 가까운 단계 선택
        for step_idx, distance in distances:
            if step_idx >= self.current_step_index and distance < 100:  # 100m 이내
                print(f"✅ 단계 {step_idx} 선택 (거리: {distance:.1f}m)")
                return step_idx
        
        # 적절한 단계를 찾지 못한 경우, 시간 기반으로 진행
        # GPS 시뮬레이션에서는 단순히 시간에 따라 단계 진행
        if self.current_gps_index > 0:
            progress_ratio = self.current_gps_index / len(SIMULATED_PATH)
            estimated_step = min(
                int(progress_ratio * len(self.route_steps)), 
                len(self.route_steps) - 1
            )
            print(f"⏰ 시간 기반 진행: 단계 {estimated_step}")
            return max(estimated_step, self.current_step_index)
        
        return self.current_step_index

    def get_navigation_guidance(self, current_lat, current_lon):
        """현재 위치 기반 네비게이션 안내 - 개선된 버전"""
        if not self.route_steps:
            return "경로 정보가 없습니다."
            
        # 현재 위치에서 적절한 단계 찾기
        new_step_index = self.find_current_step_index(current_lat, current_lon)
        
        # 단계가 진행되었는지 확인
        if new_step_index > self.current_step_index:
            self.current_step_index = new_step_index
            print(f"✅ 단계 {self.current_step_index}로 진행!")
            
        guidance = []
        guidance.append(f"📍 현재 위치: ({current_lat:.6f}, {current_lon:.6f})")
        guidance.append(f"🎯 진행 상황: {self.current_step_index + 1}/{len(self.route_steps)} 단계")
        
        # 현재 단계 안내
        if self.current_step_index < len(self.route_steps):
            current_step = self.route_steps[self.current_step_index]
            
            # 새로운 단계이거나 처음 안내하는 경우에만 "현재" 표시
            if self.current_step_index != self.last_announced_step:
                guidance.append(f"🔊 현재: {current_step}")
                self.last_announced_step = self.current_step_index
            else:
                guidance.append(f"📍 진행 중: {current_step}")
            
            # 다음 단계 미리보기
            if self.current_step_index + 1 < len(self.route_steps):
                next_step = self.route_steps[self.current_step_index + 1]
                guidance.append(f"➡️  다음: {next_step}")
            else:
                guidance.append("🎉 곧 목적지에 도착합니다!")
        else:
            guidance.append("🎉 목적지에 도착했습니다!")
            self.navigation_active = False
            
        return "\n".join(guidance)

    def simulate_gps_navigation(self):
        """GPS 시뮬레이션 및 네비게이션 안내 - 개선된 버전"""
        print("🚀 실시간 네비게이션 시작!")
        print("📱 5초마다 위치를 확인합니다...")
        print("=" * 50)
        
        while self.navigation_active and self.current_gps_index < len(SIMULATED_PATH):
            current_lat, current_lon = SIMULATED_PATH[self.current_gps_index]
            
            # 네비게이션 안내 출력
            guidance = self.get_navigation_guidance(current_lat, current_lon)
            print(f"\n[{time.strftime('%H:%M:%S')}] GPS 업데이트 #{self.current_gps_index + 1}")
            print(guidance)
            print("-" * 50)
            
            self.current_gps_index += 1
            
            if self.navigation_active and self.current_gps_index < len(SIMULATED_PATH):
                time.sleep(5)  # 5초 대기 (더 빠른 업데이트)
                
        if self.current_gps_index >= len(SIMULATED_PATH):
            print("\n🎉 시뮬레이션 경로 완주!")
            print("🎯 최종 목적지에 도착했습니다!")
            
        self.navigation_active = False

    def start_navigation(self):
        """네비게이션 시작"""
        if not self.route_steps:
            print("❌ 경로 정보가 없습니다.")
            return
            
        self.navigation_active = True
        self.current_step_index = 0
        self.current_gps_index = 0
        self.last_announced_step = -1
        
        print(f"\n📊 경로 정보:")
        print(f"   - 총 {len(self.route_steps)}개 단계")
        print(f"   - 총 {len(self.route_coordinates)}개 좌표점")
        print(f"   - 시뮬레이션 경로: {len(SIMULATED_PATH)}개 GPS 포인트")
        
        # GPS 시뮬레이션 스레드 시작
        self.gps_thread = threading.Thread(target=self.simulate_gps_navigation)
        self.gps_thread.daemon = True
        self.gps_thread.start()
        
        try:
            # 메인 스레드에서 네비게이션 활성 상태 유지
            while self.navigation_active:
                time.sleep(1)
        except KeyboardInterrupt:
            print("\n⏹️  네비게이션 중단")
            self.navigation_active = False

    def run_navigation_demo(self):
        """네비게이션 데모 실행"""
        print("🗺️  실시간 네비게이션 시스템")
        print("=" * 40)
        
        start_name = input("출발지: ").strip()
        end_name = input("도착지: ").strip()
        mode = input("이동방식 (1.대중교통/2.도보): ").strip()

        # 좌표 변환
        start_lat, start_lon = self.address_to_coord_google(start_name)
        end_lat, end_lon = self.address_to_coord_google(end_name)

        if not start_lat or not end_lat:
            print("❌ 주소 변환 실패")
            return

        print(f"\n📍 경로 계산 중...")
        
        # 경로 계산
        if mode == "1":
            self.route_steps, self.route_coordinates = self.get_combined_route_with_coordinates(
                start_lat, start_lon, end_lat, end_lon, start_name, end_name
            )
        elif mode == "2":
            self.route_steps, self.route_coordinates = self.get_pedestrian_route_with_coordinates(
                start_lat, start_lon, end_lat, end_lon, start_name, end_name
            )
        else:
            print("❌ 잘못된 입력")
            return

        if not self.route_steps:
            print("❌ 경로 계산 실패")
            return
            
        print(f"✅ 경로 계산 완료! ({len(self.route_steps)}개 단계)")
        
        # 네비게이션 시작
        self.start_navigation()


# GPS 센서 연동을 위한 확장 가능한 구조
class GPSManager:
    """실제 GPS 센서 연동시 사용할 클래스"""
    
    def __init__(self):
        self.gps_enabled = False
        
    def get_current_location(self):
        """실제 GPS에서 현재 위치 가져오기"""
        # 실제 구현시 GPS 센서 연동 코드 추가
        # 예: import gpsd, serial 등 사용
        if self.gps_enabled:
            # return actual_gps_lat, actual_gps_lon
            pass
        else:
            # 시뮬레이션 모드
            return None, None
            
    def enable_gps(self):
        """GPS 센서 활성화"""
        # 실제 GPS 센서 초기화 코드
        self.gps_enabled = True



In [ ]:


# 실행
if __name__ == "__main__":
    # API 키 설정
    google_api_key = "abcdef1234567890abcdef1234"
    tmap_api_key = "abcdef1234567890abcdef1234"

    nav_system = NavigationSystem(google_api_key, tmap_api_key)
    nav_system.run_navigation_demo()

🗺️  실시간 네비게이션 시스템

📍 경로 계산 중...
✅ 경로 계산 완료! (32개 단계)

📊 경로 정보:
   - 총 32개 단계
   - 총 32개 좌표점
   - 시뮬레이션 경로: 5개 GPS 포인트
🚀 실시간 네비게이션 시작!
📱 5초마다 위치를 확인합니다...
🔍 현재 위치: (37.623955, 127.061509)
🔍 현재 단계 인덱스: 0
   단계 0: 거리 724.7m
   단계 1: 거리 736.5m
   단계 2: 거리 737.2m
   단계 3: 거리 750.3m

[21:19:34] GPS 업데이트 #1
📍 현재 위치: (37.623955, 127.061509)
🎯 진행 상황: 1/32 단계
🔊 현재: 광운대학교 누리관부터 서울시립대학교까지 도보로 이동합니다. 안내를 시작합니다.
➡️  다음: 2m 이동
--------------------------------------------------
🔍 현재 위치: (37.618532, 127.058462)
🔍 현재 단계 인덱스: 0
   단계 0: 거리 379.6m
   단계 1: 거리 381.2m
   단계 2: 거리 383.6m
   단계 3: 거리 390.1m
⏰ 시간 기반 진행: 단계 6
✅ 단계 6로 진행!

[21:19:39] GPS 업데이트 #2
📍 현재 위치: (37.618532, 127.058462)
🎯 진행 상황: 7/32 단계
🔊 현재: 광성당 에서 좌회전 후 장위로 을 따라 7m 이동
➡️  다음: 횡단보도 후 보행자도로 을 따라 16m 이동
--------------------------------------------------
🔍 현재 위치: (37.615391, 127.057862)
🔍 현재 단계 인덱스: 6
   단계 0: 거리 606.5m
   단계 1: 거리 600.9m
   단계 2: 거리 603.2m
   단계 3: 거리 602.3m
   단계 4: 거리 594.2m
   단계 5: 거리 232.7m
   단계 6: 거리 222.0m
   단계 7

# TMAP 대중교통 API

In [59]:
import requests
import json

# 주소를 위경도로 변환 (Tmap POI 기반)
def get_coordinates_from_tmap(address, tmap_api_key):
    url = "https://apis.openapi.sk.com/tmap/pois"
    headers = {"appKey": tmap_api_key}
    params = {
        "searchKeyword": address,
        "resCoordType": "WGS84GEO",
        "reqCoordType": "WGS84GEO",
        "count": 1
    }
    res = requests.get(url, headers=headers, params=params).json()
    pois = res.get("searchPoiInfo", {}).get("pois", {}).get("poi", [])
    if not pois:
        raise ValueError(f"❌ 장소 '{address}'를 찾을 수 없습니다.")
    poi = pois[0]
    return float(poi["noorLat"]), float(poi["noorLon"])

# 도보 경로 안내
def get_tmap_walk_path(start_lat, start_lng, end_lat, end_lng, tmap_api_key):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1"
    headers = {"Content-Type": "application/json", "appKey": tmap_api_key}
    body = {
        "startX": str(start_lng),
        "startY": str(start_lat),
        "endX": str(end_lng),
        "endY": str(end_lat),
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "startName": "출발지",
        "endName": "도착지"
    }
    res = requests.post(url, headers=headers, data=json.dumps(body)).json()
    steps = []
    for feature in res.get("features", []):
        desc = feature["properties"].get("description", "")
        if desc:
            steps.append(desc)
    return steps

# 대중교통 경로 API (주소 기반, 내부 POI 자동 해석)
def get_tmap_transit_route(start_addr, end_addr, tmap_api_key):
    url = "https://apis.openapi.sk.com/transit/routes"
    headers = {"appKey": tmap_api_key, "Content-Type": "application/json"}
    body = {
        "startName": start_addr,
        "endName": end_addr,
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "searchType": "0",
        "format": "json"
    }
    res = requests.post(url, headers=headers, data=json.dumps(body)).json()
    if "metaData" not in res:
        msg = res.get("error", {}).get("message") or json.dumps(res, ensure_ascii=False)
        raise ValueError(f"❌ 대중교통 API 오류: {msg}")
    return res["metaData"]["plan"]


In [ ]:

TMAP_API_KEY = "abcdef1234567890abcdef1234"


# 📍 출발지/도착지 입력
start_address = input("출발지를 입력하세요 (예: 광운대 누리관): ")
end_address = input("도착지를 입력하세요 (예: 석계역): ")

# 🚶 이동 방식 선택
print("\n이동 방식을 선택하세요:")
print("1. 도보")
print("2. 대중교통 (버스만)")
mode = None
while mode not in [1, 2]:
    try:
        mode = int(input("선택 (1 또는 2): "))
    except:
        print("숫자 1 또는 2를 입력해주세요.")

# ✅ 실행
try:
    if mode == 1:
        # 도보 경로 안내
        start_lat, start_lng = get_coordinates_from_tmap(start_address, TMAP_API_KEY)
        end_lat, end_lng = get_coordinates_from_tmap(end_address, TMAP_API_KEY)
        steps = get_tmap_walk_path(start_lat, start_lng, end_lat, end_lng, TMAP_API_KEY)
        print(f"\n📍 {start_address}부터 {end_address}까지 도보 경로 안내를 시작합니다.")
        print(f"총 예상 시간: 약 {len(steps)*0.5:.0f}분\n")
        for i, step in enumerate(steps, 1):
            print(f"{i}. {step}")

    else:
        # 대중교통 경로 안내
        plan = get_tmap_transit_route(start_address, end_address, TMAP_API_KEY)
        total_time = plan["totalTime"] // 60
        print(f"\n📍 {start_address}부터 {end_address}까지 대중교통 경로 안내를 시작합니다.")
        print(f"총 소요시간은 약 {total_time}분입니다.\n")

        for path in plan["itineraries"]:
            for leg in path["legs"]:
                mode_type = leg["mode"]
                if mode_type == "SUBWAY":
                    continue  # 🚫 지하철 무시

                if mode_type == "WALK":
                    desc = leg.get("description", "도보 구간")
                    dur = leg.get("sectionTime", 0)
                    print(f"🚶‍♀️ 도보 약 {dur}분: {desc}")

                elif mode_type == "BUS":
                    bus = leg.get("route", "알 수 없음")
                    start_stop = leg["start"]["name"]
                    end_stop = leg["end"]["name"]
                    dur = leg.get("sectionTime", 0)
                    print(f"🚌 {start_stop} 정류장에서 {bus}번 버스 탑승 → {end_stop} 하차 (약 {dur}분)")

            break  # 추천 경로 1개만 사용

except ValueError as e:
    print(str(e))




이동 방식을 선택하세요:
1. 도보
2. 대중교통 (버스만)
❌ 대중교통 API 오류: Limit Exceeded
